# Step 1. Get data from website to dataframe

#### Get data from *https://earthquake.tenki.jp/bousai/earthquake/entries/* by Time, Location, Magnitude, Maximum

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

In [ ]:
DATALINK = 'https://earthquake.tenki.jp/bousai/earthquake/entries/'

df = pd.DataFrame(columns=['Time', 'Location', 'Magnitude', 'Maximum'])

for i in range(1,269):
    if i > 1:
        datalink = DATALINK + 'page-' + str(i) +'.html'
    else:
        datalink = DATALINK
    source = requests.get(datalink).text
    soup = BeautifulSoup(source, 'html.parser')
    table = soup.find_all('table', {'class':'earthquake-entries-table'})[0]
    trs = table.find_all('tr')[1:]
    for j in range(0,len(trs)):
        tds = trs[j].text.split('\n')
        time = tds[2]
        location = tds[3]
        magnitude = tds[4][1:]
        try:
            tmp1 = float(magnitude)
            magnitude = tmp1
        except ValueError:
            magnitude = np.nan
        if len(trs[j].find_all('td')[4].find_all('img', alt=True)) > 0:
            maximum = trs[j].find_all('td')[4].find_all('img', alt=True)[0]['alt']
            try:
                tmp2 = int(maximum)
                maximum = tmp2
            except ValueError:
                try:
                    tmp3 = int(maximum[0])
                    maximum = tmp3
                except ValueError:
                    maximum = np.nan
        else:
            maximum = np.nan
        df = df.append({'Time': time,
                        'Location': location,
                        'Magnitude': magnitude,
                        'Maximum':maximum}, ignore_index=True)
df.head(5)

## Step 2. Get all data Maximum that has greater than 4

In [34]:
df.dropna(axis=0, inplace=True)
df['Maximum'] = df.Maximum.astype(int)
df_4up = df.loc[df['Maximum'] >= 4]
df_4up.shape

(897, 4)